In [44]:
import base64
import requests
import pandas as pd
import numpy as np
import math
from datetime import date, datetime
from dateutil import tz


# Allow us to see more clearly the output
pd.set_option('float_format', '{:f}'.format)


In [3]:
# Here is the basic request if we know the lat and longitude
# We can dynamically generate this endpoint to get this data
response = requests.get(
    "https://api.sunrisesunset.io/json?lat=38.907192&lng=-77.036873&timezone=UTC&date=today")

print(response._content)



b'{"results":{"sunrise":"11:49:37 AM","sunset":"10:56:09 PM","first_light":"10:20:52 AM","last_light":"12:24:54 AM","dawn":"11:22:41 AM","dusk":"11:23:05 PM","solar_noon":"5:22:53 PM","golden_hour":"10:19:52 PM","day_length":"11:6:32","timezone":"UTC"},"status":"OK"}'


In [ ]:
# As we can see from the response from that API, we do not know the position of the sun. I found this site:
# https://gml.noaa.gov/grad/solcalc/calcdetails.html which has an excel sheet that takes the following inputs:
# latitude, longitude, time zone and date. I suggest we adapt this to a python script that can take those as inputs
# and then we can return the sunrise, sunset, and solar position without talking to an API.

# The rest of this script is converting the math that they used and recreate the calculator

In [46]:
# Inputs from the user. TODO Probably will come via frontend HTTP request so will need to be adapted
Date = pd.to_datetime('6/21/2010')
latitude = 40
longitude = -105
time_zone_object = tz.gettz('UTC')
time_zone=0


In [5]:
# Create an empty dataframe to populate with our important information
pd.DataFrame(columns=['Date','Time (past local midnight)'])

,Date,Time (past local midnight)


In [6]:
# Create an empty np array and set the first time to be six minutes past midnight, as they do in the spreadsheet
# 
local_times= np.empty(240, dtype=float, order='C', like=None)
local_times[0] = 0.004166666666666667

In [7]:
# Populate the local_times list with times six minutes apart
for idx, item in enumerate(local_times[1:]):
    local_times[idx+1] = local_times[idx] + (0.1/24)

In [10]:
# add local_time to our df
df = pd.DataFrame(local_times, columns=["local_time"])
local_times


array([0.00416667, 0.00833333, 0.0125    , 0.01666667, 0.02083333,
       0.025     , 0.02916667, 0.03333333, 0.0375    , 0.04166667,
       0.04583333, 0.05      , 0.05416667, 0.05833333, 0.0625    ,
       0.06666667, 0.07083333, 0.075     , 0.07916667, 0.08333333,
       0.0875    , 0.09166667, 0.09583333, 0.1       , 0.10416667,
       0.10833333, 0.1125    , 0.11666667, 0.12083333, 0.125     ,
       0.12916667, 0.13333333, 0.1375    , 0.14166667, 0.14583333,
       0.15      , 0.15416667, 0.15833333, 0.1625    , 0.16666667,
       0.17083333, 0.175     , 0.17916667, 0.18333333, 0.1875    ,
       0.19166667, 0.19583333, 0.2       , 0.20416667, 0.20833333,
       0.2125    , 0.21666667, 0.22083333, 0.225     , 0.22916667,
       0.23333333, 0.2375    , 0.24166667, 0.24583333, 0.25      ,
       0.25416667, 0.25833333, 0.2625    , 0.26666667, 0.27083333,
       0.275     , 0.27916667, 0.28333333, 0.2875    , 0.29166667,
       0.29583333, 0.3       , 0.30416667, 0.30833333, 0.3125 

In [45]:

my_date = datetime(2010,6,21,tzinfo=time_zone_object)
my_date = my_date.toordinal()

In [ ]:
# 6/21/2010

In [1]:
# from datetime import datetime, timedelta


# def datetime_range(start, end, delta):
#     current = start
#     while current < end:
#         yield current
#         current += delta

# dts = [pd.to_datetime(dt.strftime('%Y-%m-%d T%H:%M Z')) for dt in 
#        datetime_range(datetime(2010, 6, 21, 0), datetime(2010, 6, 21, 9+12), 
#        timedelta(minutes=6))]

# print(dts)

In [16]:
# Conver date to Julian Day
# =D2+2415018.5+E2-$B$5/24
julian_day_list = [(my_date + 1721424.5 +lt) for lt in local_times]
julian_day_list

[2455368.504166667,
 2455368.5083333333,
 2455368.5125,
 2455368.5166666666,
 2455368.5208333335,
 2455368.525,
 2455368.529166667,
 2455368.533333333,
 2455368.5375,
 2455368.5416666665,
 2455368.5458333334,
 2455368.55,
 2455368.5541666667,
 2455368.558333333,
 2455368.5625,
 2455368.566666667,
 2455368.5708333333,
 2455368.575,
 2455368.5791666666,
 2455368.5833333335,
 2455368.5875,
 2455368.591666667,
 2455368.595833333,
 2455368.6,
 2455368.6041666665,
 2455368.6083333334,
 2455368.6125,
 2455368.6166666667,
 2455368.620833333,
 2455368.625,
 2455368.629166667,
 2455368.6333333333,
 2455368.6375,
 2455368.6416666666,
 2455368.6458333335,
 2455368.65,
 2455368.654166667,
 2455368.658333333,
 2455368.6625,
 2455368.6666666665,
 2455368.6708333334,
 2455368.675,
 2455368.6791666667,
 2455368.683333333,
 2455368.6875,
 2455368.691666667,
 2455368.6958333333,
 2455368.7,
 2455368.7041666666,
 2455368.7083333335,
 2455368.7125,
 2455368.716666667,
 2455368.720833333,
 2455368.725,
 245

In [17]:
df['Julian Day'] = np.float64(julian_day_list)

In [18]:
# Find Julian Century
df['Julian Century'] = (df['Julian Day'] - 2451545.0) / 36525.0

In [22]:
#The following sections are conversions of the excel formulas to calculate the other values in the table
df['Geom Mean Long Sun (deg)'] = (280.46646 + df['Julian Century'] * (
    36000.76983 + df['Julian Century']*0.0003032
    )) % 360
df['Geom Mean Anom Sun (deg)'] = (357.52911 + df['Julian Century'] * (
    35999.05029 - 0.0001537 * df['Julian Century']))
df['Eccent Earth Orbit'] = 0.016708634 - df['Julian Century'] * (
    0.000042037+0.0000001267*df['Julian Century'])
df['Sun Eq of Ctr'] = np.sin(
    np.radians(df['Geom Mean Anom Sun (deg)'])
    ) * (
    1.914602-df['Julian Century']*(0.004817+0.000014*df['Julian Century'])
    ) + np.sin(
    np.radians( 2 * df['Geom Mean Anom Sun (deg)'])
    ) * (
    0.019993 - 0.000101 * df['Julian Century']
    ) + np.sin(
    np.radians(3*df['Geom Mean Anom Sun (deg)'])
    ) * 0.000289
df['Sun True Long (deg)'] = df['Geom Mean Long Sun (deg)'] + df['Sun Eq of Ctr']
df['Sun True Anom (deg)'] = df['Geom Mean Anom Sun (deg)'] + df['Sun Eq of Ctr']
df['Sun Rad Vector (AUs)'] = (
    1.000001018 * (
    1 - df['Eccent Earth Orbit'] * df['Eccent Earth Orbit'])
) / (
    1 + df['Eccent Earth Orbit']*np.cos(np.radians(df['Sun True Anom (deg)'])))
df['Sun App Long (deg)'] = df['Sun True Long (deg)'] - 0.00569 - 0.00478\
    * np.sin(
    np.radians(125.04 - 1934.136 * df['Julian Century'])
    )
df['Mean Obliq Ecliptic (deg)'] = 23 + (
    26 + (
    (21.448 - df['Julian Century'] * (
    46.815 + df['Julian Century'] * (
    0.00059 - df['Julian Century'] * 0.001813))))
    / 60)/60

df['Obliq Corr (deg)'] = df['Mean Obliq Ecliptic (deg)'] + 0.00256 * np.cos(
    np.radians(125.04 - 1934.136 * df['Julian Century']))

# some of the operations need to be applied to the df as a lambda, they are only used once
# thats why I they are declared the functions here. Could be optimized
def calc_sun_rt_acen_deg(sun_app_long, oblique):
    return np.degrees(
        math.atan2(
            (np.cos(np.radians(oblique))*np.sin(np.radians(sun_app_long))),(np.cos(np.radians(sun_app_long)))))

df['Sun Rt Ascen (deg)'] = df.apply(
    lambda x: calc_sun_rt_acen_deg(x['Sun App Long (deg)'], x['Obliq Corr (deg)']), axis=1)

def calc_sun_decline_deg(obliq, sun_app):
    return np.degrees(math.asin(np.sin(np.radians(obliq))*np.sin(np.radians(sun_app))))

df['Sun Declin (deg)'] = df.apply(
    lambda x: calc_sun_decline_deg(x['Obliq Corr (deg)'], x['Sun App Long (deg)']), axis=1)

df['var y'] = np.tan(np.radians(df['Obliq Corr (deg)'] / 2)) * np.tan(np.radians(df['Obliq Corr (deg)'] / 2))

df['Eq of Time (minutes)'] = 4 * np.degrees(
    df['var y'] * np.sin(
    2 * np.radians(
    df['Geom Mean Long Sun (deg)'])
    ) - 2 * df['Eccent Earth Orbit'] * np.sin(
    np.radians(
    df['Geom Mean Anom Sun (deg)'])
    ) + 4 * df['Eccent Earth Orbit'] * df['var y'] * np.sin(np.radians(df['Geom Mean Anom Sun (deg)']))
    * np.cos(2 * np.radians(df['Geom Mean Long Sun (deg)'])) - 0.5 * df['var y'] * df['var y']
    * np.sin(
    4 * np.radians(df['Geom Mean Long Sun (deg)'])
    ) - 1.25 * df['Eccent Earth Orbit'] * df['Eccent Earth Orbit'] * np.sin(
    2 * np.radians(df['Geom Mean Anom Sun (deg)'])))

def calc_ha_sunrise_deg(latitude, sun_decline):
    return np.degrees(
        math.acos(
        np.cos(np.radians(90.833)) / (np.cos(np.radians(latitude)
        ) * np.cos(np.radians(sun_decline))) - np.tan(
        np.radians(latitude)) * np.tan(np.radians(sun_decline))))

df['HA Sunrise (deg)' ] = df.apply(lambda x: calc_ha_sunrise_deg(latitude, x['Sun Declin (deg)']),axis=1)

df['Solar Noon (LST)'] = (720 - 4 * longitude - df['Eq of Time (minutes)'] + time_zone * 60)/ 1440

df['Sunrise Time (LST)'] = df['Solar Noon (LST)'] - df['HA Sunrise (deg)'] * 4 / 1440

df['Sunset Time (LST)'] = df['Solar Noon (LST)'] + df['HA Sunrise (deg)'] * 4 / 1440

df['Sunlight Duration'] = 8 * df['HA Sunrise (deg)']

df['True Solar Time (min)' ] = df['local_time'] * 1440 + df['Eq of Time (minutes)'] + 4\
      * longitude - 60 * time_zone % 1440

def calc_hour_angle_deg(true_solar_time):
    if true_solar_time / 4 < 0:
        return true_solar_time / 4 + 180
    else: 
        return true_solar_time / 4 - 180



    




In [24]:
df['Hour Angle (deg)'] = df.apply(
    lambda x: calc_hour_angle_deg(x['True Solar Time (min)']), axis=1)


In [26]:
def calc_solar_zeith(latitude, sun_declin_deg, hour_angle):
    return np.degrees(
    math.acos(np.sin(np.radians(latitude)) * np.sin(
    np.radians(sun_declin_deg)) + np.cos(np.radians(latitude)) * np.cos(
    np.radians(sun_declin_deg)) * np.cos(np.radians(hour_angle))))

In [29]:
df['Solar Zenith Angle (deg)'] = df.apply(
    lambda x: calc_solar_zeith(latitude, x['Sun Declin (deg)'], x['Hour Angle (deg)']), axis=1)
    


In [32]:
df['Solar Elevation Angle (deg)'] = 90 - df['Solar Zenith Angle (deg)']

In [30]:
# IF(AE2>85,0,IF(AE2>5,58.1/TAN(RADIANS(AE2))-0.07/POWER(TAN(RADIANS(AE2)),3)+0.000086/POWER(TAN(RADIANS(AE2)),5),IF(AE2>-0.575,1735+AE2*(-518.2+AE2*(103.4+AE2*(-12.79+AE2*0.711))),-20.772/TAN(RADIANS(AE2))))
#     )
# /3600

def calc_approx_atm(solar_elevation):
    if solar_elevation > 85:
        temp =  0
    elif solar_elevation > 5:
        temp =  58.1/np.tan(np.radians(solar_elevation))-0.07/np.power(np.tan(np.radians(solar_elevation)),3)\
            + 0.000086 / np.power(np.tan(np.radians(solar_elevation)),5)
    elif solar_elevation > -0.575:
        temp =  1735+solar_elevation*(-518.2+solar_elevation*(103.4+solar_elevation*(-12.79+solar_elevation*0.711)))
    else:
        temp =  -20.772/np.tan(np.radians(solar_elevation))
    return temp / 3600


In [33]:
df['Approx Atmospheric Refraction (deg)'] = df.apply(
    lambda x: calc_approx_atm(x['Solar Elevation Angle (deg)']), axis=1) 

In [35]:
df['Solar Elevation corrected for atm refraction (deg)'] = df['Solar Elevation Angle (deg)'] + df['Approx Atmospheric Refraction (deg)']

In [ ]:
# =IF(AC2>0,
#     MOD(DEGREES(ACOS(((SIN(RADIANS($B$3))*COS(RADIANS(AD2)))-SIN(RADIANS(T2)))/(COS(RADIANS($B$3))*SIN(RADIANS(AD2)))))+180,360),
#                                    MOD(540-DEGREES(ACOS(((SIN(RADIANS($B$3))*COS(RADIANS(AD2)))-SIN(RADIANS(T2)))/(COS(RADIANS($B$3))*SIN(RADIANS(AD2))))),360))

=IF(AC2>0,MOD(DEGREES(ACOS(((SIN(RADIANS($B$3))*COS(RADIANS(AD2)))-SIN(RADIANS(T2)))/(COS(RADIANS($B$3))*SIN(RADIANS(AD2)))))+180,360),MOD(540-DEGREES(ACOS(((SIN(RADIANS($B$3))*COS(RADIANS(AD2)))-SIN(RADIANS(T2)))/(COS(RADIANS($B$3))*SIN(RADIANS(AD2))))),360))


In [38]:
def calc_azimuth_angle(sun_decline, hour_angle, solar_zenith_angle):
    if hour_angle>0:
           return np.degrees(
                math.acos(
                ((np.sin(np.radians(latitude))*np.cos(np.radians(solar_zenith_angle)))-np.sin(np.radians(sun_decline))
                 )/(np.cos(np.radians(latitude))*np.sin(np.radians(solar_zenith_angle))))) + 180 % 360
    else:
        return 540 - np.degrees(
             math.acos(((np.sin(np.radians(latitude)) * np.cos(np.radians(solar_zenith_angle))
                         ) - np.sin(np.radians(sun_decline))
                         ) / 
                         (np.cos(np.radians(latitude)) * np.sin(np.radians(solar_zenith_angle))))) % 360



In [39]:
df['Solar Azimuth Angle (deg cw from N)'] = df.apply(
    lambda x: calc_azimuth_angle(x['Sun Declin (deg)'],x['Hour Angle (deg)'], x['Solar Zenith Angle (deg)']), axis=1) 

In [40]:
df

,local_time,Julian Day,Julian Century,Geom Mean Long Sun (deg),Geom Mean Anom Sun (deg),Eccent Earth Orbit,Sun Eq of Ctr,Sun True Long (deg),Sun True Anom (deg),Sun Rad Vector (AUs),...,Sunrise Time (LST),Sunset Time (LST),Sunlight Duration,True Solar Time (min),Hour Angle (deg),Solar Zenith Angle (deg),Solar Elevation Angle (deg),Approx Atmospheric Refraction (deg),Solar Elevation corrected for atm refraction (deg),Solar Azimuth Angle (deg cw from N)
0,0.004167,2455368.504167,0.104682,89.093252,4125.975892,0.016704,0.454641,89.547893,4126.430533,1.016223,...,0.480009,1.105619,900.877524,-415.651935,76.087016,64.870442,25.129558,0.034219,25.163778,280.365105
1,0.008333,2455368.508333,0.104682,89.097359,4125.979999,0.016704,0.454511,89.551869,4126.434509,1.016224,...,0.480010,1.105619,900.877642,-409.652841,77.586790,65.998973,24.001027,0.036028,24.037055,281.230796
2,0.012500,2455368.512500,0.104682,89.101466,4125.984105,0.016704,0.454380,89.555845,4126.438485,1.016224,...,0.480010,1.105620,900.877759,-403.653748,79.086563,67.124129,22.875871,0.037994,22.913865,282.094187
3,0.016667,2455368.516667,0.104682,89.105572,4125.988212,0.016704,0.454249,89.559822,4126.442461,1.016224,...,0.480011,1.105621,900.877876,-397.654654,80.586337,68.245661,21.754339,0.040140,21.794479,282.956001
4,0.020833,2455368.520833,0.104682,89.109679,4125.992319,0.016704,0.454119,89.563798,4126.446437,1.016224,...,0.480012,1.105621,900.877991,-391.655560,82.086110,69.363321,20.636679,0.042493,20.679172,283.816932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,0.983333,2455369.483333,0.104709,90.058365,4126.940959,0.016704,0.423886,90.482251,4127.364845,1.016287,...,0.480157,1.105766,900.876636,994.135280,68.533820,59.144481,30.855519,0.026923,30.882442,275.942221
236,0.987500,2455369.487500,0.104709,90.062472,4126.945066,0.016704,0.423755,90.486226,4127.368820,1.016287,...,0.480158,1.105767,900.876509,1000.134376,70.033594,60.286239,29.713761,0.028174,29.741935,276.831636
237,0.991667,2455369.491667,0.104709,90.066579,4126.949172,0.016704,0.423623,90.490202,4127.372796,1.016288,...,0.480159,1.105767,900.876381,1006.133471,71.533368,61.425884,28.574116,0.029513,28.603628,277.714587
238,0.995833,2455369.495833,0.104709,90.070685,4126.953279,0.016704,0.423492,90.494178,4127.376771,1.016288,...,0.480159,1.105768,900.876252,1012.132567,73.033142,62.563161,27.436839,0.030948,27.467787,278.591984


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   local_time                 240 non-null    float64
 1   Julian Day                 240 non-null    float64
 2   Julian Century             240 non-null    float64
 3   Geom Mean Long Sun (deg)   240 non-null    float64
 4   Geom Mean Anom Sun (deg)   240 non-null    float64
 5   Eccent Earth Orbit         240 non-null    float64
 6   Sun Eq of Ctr              240 non-null    float64
 7   Sun True Long (deg)        240 non-null    float64
 8   Sun True Anom (deg)        240 non-null    float64
 9   Sun Rad Vector (AUs)       240 non-null    float64
 10  Sun App Long (deg)         240 non-null    float64
 11  Mean Obliq Ecliptic (deg)  240 non-null    float64
 12  Obliq Corr (deg)           240 non-null    float64
 13  Sun Rt Ascen (deg)         240 non-null    float64